# Libs

In [1]:
from deap import base, creator, tools, algorithms
import pandas as pd
import random

# Disciplinas

In [2]:
disciplinas = pd.read_excel('disciplinas.xlsx')
disciplinas['Disciplina'] = disciplinas['Disciplina'].str.upper()
disciplinas = disciplinas.groupby('Disciplina')['Créditos'].sum()
disciplinas = [(disciplina, creditos) for disciplina, creditos in disciplinas.items()]

# Professores

In [3]:
professores = pd.read_excel('professores.xlsx')
professores['Preferências'] = professores[['Disciplinas', 'Disciplinas.1', 'Disciplinas.2']].bfill(axis=1).iloc[:, 0]
professores['Preferências'] = professores['Preferências'].str.upper()
professores['Preferências'] = professores['Preferências'].str.split(';')
professores['Nome'] = professores['Nome de usuário'].str.split('@').str[0]
professores = professores.loc[:,['Nome', 'Preferências', 'CH MAX']].dropna()
professores = professores[['Nome', 'Preferências', 'CH MAX']].to_dict(orient='records')

# Alocacao

In [4]:
# Quantidade de disciplinas e professores
num_disciplinas = len(disciplinas)
num_professores = len(professores)

# Criar tipos de fitness e indivíduo
creator.create("FitnessMulti", base.Fitness, weights=(1, -1))  # Maximizar preferências e minimizar excedente de carga
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Configuração do toolbox
toolbox = base.Toolbox()

# Registro de função para inicializar um indivíduo com alocação aleatória
toolbox.register("indices", random.randint, 0, num_professores - 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.indices, n=num_disciplinas)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Função de avaliação do indivíduo
def avaliar(individual):
    carga_horaria = [0] * num_professores  # Armazena a carga horária de cada professor
    preferencias = 0  # Contador de preferências atendidas
    excedente_ch = 0  # Soma do excedente de carga horária

    # Alocação das disciplinas e cálculo das cargas horárias e preferências atendidas
    for i, professor_idx in enumerate(individual):
        carga_horaria[professor_idx] += disciplinas[i][1]
        if disciplinas[i][0] in professores[professor_idx]['Preferências']:
            preferencias += 1

    # Calcula o excedente de carga horária para professores que excedem seu CH MAX
    for i, ch in enumerate(carga_horaria):
        if ch > professores[i]['CH MAX']:
            excedente_ch += ch - professores[i]['CH MAX']

    return 100*preferencias/num_disciplinas, excedente_ch  # Retorna percentual de preferências e excedente de carga horária

# Registro dos operadores genéticos
toolbox.register("mate", tools.cxUniform, indpb=0.5)#toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=num_professores - 1, indpb=0.2)
toolbox.register("select", tools.selNSGA2)
toolbox.register("evaluate", avaliar)

# Configuração das estatísticas do algoritmo
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg_pref", lambda fits: sum(fit[0] for fit in fits) / len(fits))  # Média de preferências atendidas
stats.register("avg_excedente", lambda fits: sum(fit[1] for fit in fits) / len(fits))  # Média do excedente de carga horária

# Parâmetros do algoritmo genético
populacao = toolbox.population(n=100000)
algorithms.eaMuPlusLambda(populacao, toolbox, mu=20000, lambda_=60000, cxpb=0.7, mutpb=0.1, ngen=100, stats=stats, verbose=True)

# Melhor resultado
melhor_individuo = tools.selBest(populacao, k=1)[0]

gen	nevals	avg_pref	avg_excedente
0  	100000	21.6696 	51.8185      
1  	47896 	24.5801 	37.1006      
2  	47981 	26.342  	33.3582      
3  	48248 	28.5336 	31.0633      
4  	47854 	30.4435 	29.2497      
5  	48039 	31.6871 	27.645       
6  	47894 	33.0787 	26.4363      
7  	48054 	34.1734 	25.3638      
8  	48030 	35.6395 	24.5818      
9  	47878 	38.5774 	24.5831      
10 	47940 	38.4335 	23.2905      
11 	48070 	40.9999 	23.1567      
12 	48088 	41.6202 	22.2281      
13 	47912 	42.8535 	21.6321      
14 	48089 	43.9026 	21.103       
15 	48011 	44.3882 	20.357       
16 	48079 	45.3521 	19.9386      
17 	47978 	46.402  	19.6576      
18 	48056 	49.864  	20.234       
19 	48045 	49.3327 	19.3017      
20 	47971 	50.53   	19.1546      
21 	47935 	49.8364 	18.2079      
22 	47961 	53.1003 	18.8468      
23 	47992 	52.9688 	18.0095      
24 	48020 	51.9267 	16.9002      
25 	47920 	56.4233 	17.2645      
26 	48014 	55.9062 	16.2935      
27 	47903 	56.2878 	16.4035      
28 	48186 	58.

# Resultado

In [5]:
# Listar professores com suas disciplinas e carga horária alocadas
alocacao_final = {professor['Nome']: {'disciplinas': [], 'carga_horaria': 0} for professor in professores}

# Distribuir as disciplinas entre os professores conforme o melhor indivíduo
for i, professor_idx in enumerate(melhor_individuo):
    professor_nome = professores[professor_idx]['Nome']
    alocacao_final[professor_nome]['disciplinas'].append(disciplinas[i][0])
    alocacao_final[professor_nome]['carga_horaria'] += disciplinas[i][1]
    
# Transformar o dicionário em uma lista de tuplas para criar o DataFrame
dados_alocacao = []
for professor, dados in alocacao_final.items():
    disciplinas_alocadas = ', '.join(dados['disciplinas'])  # Juntar as disciplinas em uma única string
    carga_horaria_total = dados['carga_horaria']
    dados_alocacao.append((professor, disciplinas_alocadas, carga_horaria_total))

# Criar o DataFrame
df_alocacao = pd.DataFrame(dados_alocacao, columns=['Professor', 'Disciplinas Alocadas', 'Carga Horária Total (h.a)'])

# Salvar o DataFrame
df_alocacao.to_excel('alocacao.xlsx')

# Exibir o DataFrame
df_alocacao

,Professor,Disciplinas Alocadas,Carga Horária Total (h.a)
0,adriano.franca,"ARQUITETURA DE COMPUTADORES, CABEAMENTO ESTRUT...",13
1,ricson.santana,"LÓGICA DE PROGRAMAÇÃO, PROJETO DE REDES",16
2,valdir.silva,"ADMINISTRAÇÃO DE SISTEMAS OPERACIONAIS, ENGENH...",16
3,delano.oliveira,"PROGRAMAÇÃO ORIENTADA A OBJETOS, PROJETO DE DE...",13
4,victor.cavalcanti,"DESENVOLVIMENTO WEB, LINGUAGEM E TÉCNICAS DE P...",18
5,thiago.bezerra,"REDES SEM FIO , SEGURANÇA EM REDES DE COMPUTAD...",16
6,saulomarques.morais,"APLICATIVOS COMERCIAIS II, INFORMÁTICA BÁSICA,...",22
7,leonardo.lima,"BANCO DE DADOS II, PROJETO DE DESENVOLVIMENTO ...",18
8,tiago.lima,"INTELIGÊNCIA ARTIFICIAL, METODOLOGIA CIENTÍFIC...",15
9,david.ribeiro,"FUNDAMENTOS DE COMPUTAÇÃO CONCORRENTE, PARALEL...",16
